In [1]:
import numpy as np

In [2]:
# Naive uncorrealted chi2
def naive(data):
    return np.sum(data ** 2, axis=-1)

In [3]:
# Calculate "chi2" w/ fitted correlation
def fitted(data):
    return np.max(data ** 2, axis=-1)

In [4]:
# Define expected distributions for fitted "chi2"
from scipy.stats import norm, chi, chi2, rv_continuous
from scipy.special import erf, erfinv


class Bee(rv_continuous):
    def _cdf(self, x, df):
        return erf(x / np.sqrt(2)) ** df

    def _pdf(self, x, df):
        return (
            df
            * (erf(x / np.sqrt(2))) ** (df - 1)
            * np.sqrt(2 / np.pi)
            * np.exp(-(x ** 2) / 2)
        )

    def ppf(self, x, df):
        return erfinv((x) ** (1 / df)) * np.sqrt(2)


# Instance of the distribution, support starts at 0
BEE = Bee(a=0)


class Bee2(rv_continuous):
    def _cdf(self, x, df):
        b = np.sqrt(x)
        ret = BEE.cdf(b, df)
        return ret

    def _pdf(self, x, df):
        return (
            df
            * (erf(np.sqrt(x / 2))) ** (df - 1)
            / np.sqrt(2 * np.pi * x)
            * np.exp(-x / 2)
        )

    def _ppf(self, x, df):
        b = BEE.ppf(x, df)
        return b ** 2


# Instance of the distribution, support starts at 0
BEE2 = Bee2(a=0)

In [5]:
# Transorm uncorrelated fitted "chi2" to chi2(ndof=1)
def scaled(data, N_dim):
    return 2 * erfinv(erf(np.sqrt(fitted(data)) / np.sqrt(2)) ** N_dim) ** 2
    # return (2/np.pi)**(N_dim-1) * x**N_dim

In [6]:
# Approximately invariant measures
from scipy.special import logsumexp

# from numpy.linalg import norm as nrm
from scipy.optimize import root_scalar
from functools import lru_cache


def p_mean(x, p):
    N = x.shape[-1]
    return np.exp(1 / p * (logsumexp(p * np.log(x), axis=-1) - np.log(N)))


def p_norm(x, p):
    return np.exp(1 / p * (logsumexp(p * np.log(x), axis=-1)))


def smooth_min(x, p):
    return p_mean(x, -p)


def smooth_max(x, p):
    return p_mean(x, p)


def invariant1(x):
    """Triangular wings"""
    cdf = chi2.cdf(x ** 2, df=1)
    # a = smooth_min(cdf, p[0])
    a = np.min(cdf, axis=-1)
    # b = smooth_max(cdf, p[1])
    b = np.max(cdf, axis=-1)

    x = a / (1 - b + a)
    return chi2.ppf(x, df=1)

In [7]:
@np.vectorize
@lru_cache(500000)
def _minfrommax(b, df=2):
    """Position of sides of missing corner from coordinates of hypercube sides."""
    # b**df - (b-x)**df = x
    def f(x):
        return b ** df - (b - x) ** df - x

    if df > 1:

        def fp(x):
            return df * (b - x) ** (df - 1) * (-1)

    else:

        def fp(x):
            return -df * np.ones_like(x)

    if df > 2:

        def fpp(x):
            return df * (df - 1) * (b - x) ** (df - 2)

    else:

        def fpp(x):
            return df * (df - 1) * np.ones_like(x)

    # return root_scalar(f, fprime=fp, fprime2=fpp, x0=0.001, x1=0.9).root
    # return root_scalar(f, fprime=fp, x0=b).root
    if b == 0:
        return 0
    else:
        return root_scalar(f, x0=b, x1=b / 2).root
    # return root_scalar(f, bracket=(0.0,b)).root


def minfrommax(b, df=2):
    """Position of sides of missing corner from coordinates of hypercube sides."""
    step = 0.0001
    b_ = np.floor(b / step, dtype=float) * step
    b__ = b_ + step
    delta = (b - b_) / step
    x_ = _minfrommax(b_, df)
    x__ = _minfrommax(b__, df)
    return x_ + (x__ - x_) * delta


def invariant2(x):
    """Hypercube minus corner"""
    cdf = chi2.cdf(x ** 2, df=1)
    # a = smooth_min(cdf, p[0])
    a = np.min(cdf, axis=-1)
    # b = smooth_max(cdf, p[1])
    b = np.max(cdf, axis=-1)

    df = x.shape[-1]
    mfm = minfrommax(b, df=df)
    x = np.maximum(a, mfm)
    # Cap value in case of rounding errors or stray root finding
    x = np.minimum(x, 1.0 - 1e-9)
    return chi2.ppf(x, df=1)

In [12]:
@np.vectorize
@lru_cache(10000)
def _yfrommax(b, df=2, alpha=0.5):
    """(1 - diagonal coordinate) from (1 - max) of accepted region"""
    # A = (1-b)**df - ((y-b)**df)/((1-alpha+alpha*y)**(df-1)) = 1 - y
    beta = 1 - alpha
    q = (1.0 - b) ** df - 1
    dfm = df - 1

    def f(y):
        return q - ((y - b) ** df) / ((beta + alpha * y) ** (dfm)) + y

    if b <= 0:
        return 0.0
    if b >= 1:
        return 1.0
    else:
        return root_scalar(f, x0=b, x1=b * 1.001).root


def yfrommax(b, df=2, alpha=0.5):
    """Buffer and interpolate values to speed things up."""
    step = 0.0001
    b_ = np.floor(b / step, dtype=float) * step
    b__ = b_ + step
    delta = (b - b_) / step
    x_ = _yfrommax(b_, df=df, alpha=alpha)
    x__ = _yfrommax(b__, df=df, alpha=alpha)
    return x_ + (x__ - x_) * delta


def invariant3(x, alpha=0.5, fast=False):
    """Return test statistic given vector of normalised values."""
    if fast:
        sf = 1 - chi2.cdf(x ** 2, df=1)  # Faster, but less accurate
    else:
        sf = chi2.sf(x ** 2, df=1)

    # Get possible diagonal coordinate from maximum CDF value (= minimum SF)
    a = np.min(sf, axis=-1)
    b = np.max(sf, axis=-1)
    yfm = yfrommax(a, df=x.shape[-1], alpha=alpha)

    # Get possible diagonal coordinate from centre surface
    yfc = (alpha * (a - b) + b) / (1.0 + alpha * (a - b))

    y = np.minimum(yfc, yfm)
    y = np.maximum(y, 0)  # Cap in case of rounding or root finding errors
    return chi2.isf(y, df=1)